In [24]:
import json

import pandas as pd

from shapely.geometry import Point
from shapely.geometry import Polygon
from CCAgT_utils.converters.LabelBox import CCAgT_Annotations

In [25]:
data = [{'image_name': 'A_xxx', 'geometry': Point(1, 1), 'category_id': 3},
        {'image_name': 'A_yyy', 'geometry': Point(10, 10), 'category_id': 3},
        {'image_name': 'A_yyy', 'geometry': Polygon([(40, 40), (50, 50), (50, 40)]), 'category_id': 1},
        {'image_name': 'B_yyy', 'geometry': Point(10, 10), 'category_id': 3},
        {'image_name': 'B_yyy', 'geometry': Point(2000, 2000), 'category_id': 3}]

df = pd.DataFrame(data)
df

,image_name,geometry,category_id
0,A_xxx,POINT (1 1),3
1,A_yyy,POINT (10 10),3
2,A_yyy,"POLYGON ((40 40, 50 50, 50 40, 40 40))",1
3,B_yyy,POINT (10 10),3
4,B_yyy,POINT (2000 2000),3


In [26]:
ccagt_ann = CCAgT_Annotations(df)

In [27]:
df['slide_id'] = ccagt_ann.get_slide_id()
df

,image_name,geometry,category_id,slide_id
0,A_xxx,POINT (1 1),3,A
1,A_yyy,POINT (10 10),3,A
2,A_yyy,"POLYGON ((40 40, 50 50, 50 40, 40 40))",1,A
3,B_yyy,POINT (10 10),3,B
4,B_yyy,POINT (2000 2000),3,B


In [28]:
df['geo_type'] = ccagt_ann.geometries_type()
df

,image_name,geometry,category_id,slide_id,geo_type
0,A_xxx,POINT (1 1),3,A,Point
1,A_yyy,POINT (10 10),3,A,Point
2,A_yyy,"POLYGON ((40 40, 50 50, 50 40, 40 40))",1,A,Polygon
3,B_yyy,POINT (10 10),3,B,Point
4,B_yyy,POINT (2000 2000),3,B,Point


In [29]:
sat_series = df.loc[(df['category_id']==3) & (df['geo_type']=='Point'), 'geometry']

df.loc[(df['category_id']==3) & (df['geo_type']=='Point'), 'geometry'] = ccagt_ann.satellite_point_to_polygon(sat_series)

df['geo_type'] = ccagt_ann.geometries_type()

df

,image_name,geometry,category_id,slide_id,geo_type
0,A_xxx,"POLYGON ((6.046265044040321 1, 5.6621409898160...",3,A,Polygon
1,A_yyy,"POLYGON ((15.04626504404032 10, 14.66214098981...",3,A,Polygon
2,A_yyy,"POLYGON ((40 40, 50 50, 50 40, 40 40))",1,A,Polygon
3,B_yyy,"POLYGON ((15.04626504404032 10, 14.66214098981...",3,B,Polygon
4,B_yyy,"POLYGON ((2005.04626504404 2000, 2004.66214098...",3,B,Polygon


In [30]:
df['geometry'] = ccagt_ann.fit_geometries_to_image_boundary()
df

,image_name,geometry,category_id,slide_id,geo_type
0,A_xxx,"POLYGON ((5.847352676660663 0, 0 0, 0 5.847352...",3,A,Polygon
1,A_yyy,"POLYGON ((15.04626504404032 10, 14.66214098981...",3,A,Polygon
2,A_yyy,"POLYGON ((40 40, 50 50, 50 40, 40 40))",1,A,Polygon
3,B_yyy,"POLYGON ((15.04626504404032 10, 14.66214098981...",3,B,Polygon
4,B_yyy,NaN,3,B,Polygon


In [31]:
df.dropna(axis=0, subset=['geometry'], inplace=True)


In [32]:
df['area'] = ccagt_ann.geometries_area()
df

,image_name,geometry,category_id,slide_id,geo_type,area
0,A_xxx,"POLYGON ((5.847352676660663 0, 0 0, 0 5.847352...",3,A,Polygon,30.383525
1,A_yyy,"POLYGON ((15.04626504404032 10, 14.66214098981...",3,A,Polygon,77.959629
2,A_yyy,"POLYGON ((40 40, 50 50, 50 40, 40 40))",1,A,Polygon,50.000000
3,B_yyy,"POLYGON ((15.04626504404032 10, 14.66214098981...",3,B,Polygon,77.959629


In [33]:
df['image_id'] = ccagt_ann.generate_ids(df['image_name'])
df

,image_name,geometry,category_id,slide_id,geo_type,area,image_id
0,A_xxx,"POLYGON ((5.847352676660663 0, 0 0, 0 5.847352...",3,A,Polygon,30.383525,1
1,A_yyy,"POLYGON ((15.04626504404032 10, 14.66214098981...",3,A,Polygon,77.959629,2
2,A_yyy,"POLYGON ((40 40, 50 50, 50 40, 40 40))",1,A,Polygon,50.000000,2
3,B_yyy,"POLYGON ((15.04626504404032 10, 14.66214098981...",3,B,Polygon,77.959629,3


In [34]:
ccagt_ann.df

,image_name,geometry,category_id,slide_id,geo_type,area,image_id
0,A_xxx,"POLYGON ((5.847352676660663 0, 0 0, 0 5.847352...",3,A,Polygon,30.383525,1
1,A_yyy,"POLYGON ((15.04626504404032 10, 14.66214098981...",3,A,Polygon,77.959629,2
2,A_yyy,"POLYGON ((40 40, 50 50, 50 40, 40 40))",1,A,Polygon,50.000000,2
3,B_yyy,"POLYGON ((15.04626504404032 10, 14.66214098981...",3,B,Polygon,77.959629,3
